In [1]:
import numpy as np
import pandas as pd
import jieba
import random
import re
from gensim.models.word2vec import LineSentence
from gensim.models import word2vec
import gensim

In [2]:
def load_data(data_path):
    data = pd.read_csv(data_path)
    data = data.fillna('')
    
#     test_data_index = random.choice(range(len(data)), len(data) // 4)
#     test_data = data[data.index.isin(test_data_index)]
#     train_data = data[~data.index.isin(test_data_index)]
#     return train_data, test_data
    return data

def load_stopwords(stopwords_path):
    return [word.strip() for word in open(stopwords_path, encoding = 'utf-8')]

def clean_sentence(sentence):
    pattern = r'\（.*?\）'
    sentence = re.sub(pattern, '', sentence)
    return sentence

def process_sentence(sentence):
    sentence = clean_sentence(sentence)
    
    stopwords = load_stopwords('stopwords.txt')
    return ' '.join([word for word in jieba.lcut(sentence) if word not in stopwords])

In [3]:
data = load_data('Movie_information.csv')
data['tokenized_description'] = data.apply(lambda x: process_sentence(x[1]) + ' ' + process_sentence(x[2]), axis = 1)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/4x/l9lh69rx2s119d4xt52d5pz00000gn/T/jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built successfully.


In [4]:
data['actor'] = data.apply(lambda x: ' '.join(x[3].split(',')), axis = 1)
data['director'] = data.apply(lambda x: ' '.join(x[4].split(',')), axis = 1)
data['screen_writer'] = data.apply(lambda x: ' '.join(x[5].split(',')), axis = 1)

In [5]:
data.head()

,id,电影名,内容简介,主演,导演,编剧,评分,预告片URL,tokenized_description,actor,director,screen_writer
0,1292900,摇啊摇，摇到外婆桥,1930年代，乡村少年唐水生来到上海投靠六叔（李雪健），在他的引荐下，做了歌舞皇后小金宝（巩...,"巩俐,李保田,李雪健,孙淳",张艺谋,毕飞宇,7.2,,摇 摇 摇 外婆桥 1930 年代 乡村 少年 唐 水生 来到 上海 投靠 六叔 引荐 歌舞...,巩俐 李保田 李雪健 孙淳,张艺谋,毕飞宇
1,34937798,杨门少年,本片以时局动荡的五代十国时期为背景，讲述了杨继业和佘太君年轻时候的故事：后汉末年，皇位空缺、...,"杨清,赵菁,战克林,李沛霖",吴远博,"吴远博,飞刀叶",0.0,,杨门 少年 本片 时局动荡 五代十国 时期 背景 讲述 杨继业 佘太君 年轻 故事 后汉 末...,杨清 赵菁 战克林 李沛霖,吴远博,吴远博 飞刀叶
2,4306001,关东民谣,新上任的青山乡党委书记秦丽璞，从村里流传的民谣中了解到领导干部的腐败现象，给村民带来的不满情...,刘欣,高天红,"高满堂,王霆钧",0.0,,关东 民谣 新 上任 青山 乡 党委书记 秦丽璞 村里 流传 民谣 领导 干部 腐败现象 村...,刘欣,高天红,高满堂 王霆钧
3,5318664,中国记忆：大将粟裕,在解放军开国将帅当中粟裕的经历颇为传奇，1927年他参加南昌起义的时候只有20岁，那时候是一...,,凤凰卫视,,0.0,,中国 记忆 大将 粟裕 解放军 开国 将帅 当中 粟裕 经历 颇为 传奇 1927 参加 南...,,凤凰卫视,
4,1433417,向日葵,本片延续了张扬在《洗澡》、《昨天》等影片中所呈现的中国式父子冲突与和解，以1976、1987...,"孙海英,陈冲,王海地,张凡",张杨,"蔡尚君,张杨,霍昕",7.6,,向日葵 本片 延续 张扬 洗澡 昨天 影片 呈现 中国式 父子 冲突 和解 1976 198...,孙海英 陈冲 王海地 张凡,张杨,蔡尚君 张杨 霍昕


In [6]:
merged_data = data.apply(lambda x: ' '.join(x[8:]), axis = 1)
merged_data = pd.concat([merged_data, data['评分']], axis = 1)
merged_data.columns = ['descriptions', 'scores']
merged_data = merged_data[merged_data['scores'] > 0]
merged_data['scores'] = merged_data.apply(lambda x: 1 if x[1] > 5 else 0, axis = 1)
merged_data.head()

,descriptions,scores
0,摇 摇 摇 外婆桥 1930 年代 乡村 少年 唐 水生 来到 上海 投靠 六叔 引荐 歌舞...,1
4,向日葵 本片 延续 张扬 洗澡 昨天 影片 呈现 中国式 父子 冲突 和解 1976 198...,1
6,囧 途之泰 囧 商业 成功人士 徐朗用 五年 时间 发明 一种 油霸 神奇 产品 — — 每...,1
7,少林寺 隋朝 末年 隋 王世充 侄子 王仁则 督建 河防 工事 杀死 反抗 暴虐 神腿 张 ...,1
9,斗地主 故事 发生 民国初年 春天 青楼 当红 头牌 无数 男人 一睹 芳泽 散尽 千金 某...,0


In [7]:
merged_data['descriptions'].to_csv('description.csv', header = None, index = False)
merged_data.to_csv('movie_info.csv')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [8]:
model_wv = word2vec.Word2Vec(LineSentence('description.csv'), sg=1,workers=8,min_count=2,size=100)

In [9]:
merged_data.head()

,descriptions,scores
0,摇 摇 摇 外婆桥 1930 年代 乡村 少年 唐 水生 来到 上海 投靠 六叔 引荐 歌舞...,1
4,向日葵 本片 延续 张扬 洗澡 昨天 影片 呈现 中国式 父子 冲突 和解 1976 198...,1
6,囧 途之泰 囧 商业 成功人士 徐朗用 五年 时间 发明 一种 油霸 神奇 产品 — — 每...,1
7,少林寺 隋朝 末年 隋 王世充 侄子 王仁则 督建 河防 工事 杀死 反抗 暴虐 神腿 张 ...,1
9,斗地主 故事 发生 民国初年 春天 青楼 当红 头牌 无数 男人 一睹 芳泽 散尽 千金 某...,0


In [10]:
model_wv.wv.vectors.shape

(30484, 100)

In [11]:
merged_data.shape

(2835, 2)

In [12]:
model_wv.save('word2vec_model')

In [13]:
def get_max_len(data):
    '''
    max_len = mean(max lenths of each entry sentence) + 2 * standard deviation
    '''
    max_lens = data.apply(lambda x: x.count(' '))
    return int(np.mean(max_lens) + 2 * np.std(max_lens))

def pad_sentence(sentence, max_len, vocab):
    '''
    <START> ...<UNK>... <END> <PAD>
    '''
    words = sentence.split()[:max_len]
    
    sentence = [word if word in vocab else '<UNK>' for word in words]
    
    sentence = ['<START>'] + sentence + ['<END>']
    sentence = sentence + ['<PAD>'] * (max_len + 2 - len(words))
    return ' '.join(sentence)

In [14]:
vocab = model_wv.wv.vocab
max_len = get_max_len(merged_data['descriptions'])
print('max: {}'.format(max_len))
merged_data['descriptions'] = merged_data.apply(lambda x: pad_sentence(x[0], max_len, vocab), axis = 1)

max: 349


In [15]:
merged_data.head()

,descriptions,scores
0,<START> 摇 摇 摇 外婆桥 1930 年代 乡村 少年 唐 水生 来到 上海 投靠 ...,1
4,<START> 向日葵 本片 延续 张扬 洗澡 昨天 影片 呈现 中国式 父子 冲突 和解 ...,1
6,<START> 囧 <UNK> 囧 商业 <UNK> <UNK> 五年 时间 发明 一种 油...,1
7,<START> 少林寺 <UNK> 末年 隋 <UNK> 侄子 王仁则 <UNK> <UNK...,1
9,<START> 斗地主 故事 发生 民国初年 春天 青楼 当红 头牌 无数 男人 <UNK>...,0


In [16]:
merged_data['descriptions'].to_csv('padded_descriptions.csv', index = False, header = None)
merged_data.to_csv('processed_data.csv')
print('begin retraining word2vec model')
model_wv.build_vocab(LineSentence('padded_descriptions.csv'), update=True)
model_wv.train(LineSentence('padded_descriptions.csv'), epochs = 2, total_examples = model_wv.corpus_count)

model_wv.save('pad_word2vec_model')
print('finish retraining w2v model')
print('final w2v_model has vocabulary of ', len(model_wv.wv.vocab))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


begin retraining word2vec model
finish retraining w2v model
final w2v_model has vocabulary of  30490
